In [17]:
#7_00PM is the best model to use

In [1]:
from baseline_copy2 import *

In [5]:
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model)
batch_size = 64
path1 = 'saved_models/Nov15/06_22PM.pt'
path = 'lastModel.pt'
fold1 = 'fold1.pt'
fold2 = 'fold2.pt'

model = torch.load(fold2)

device = torch.device('cuda:0')
# with torch.cuda.device(dev0):
#     t = t.cuda()
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model)


model.to(device)

#enter eval mode
model.eval()

def calculate_accu(outputs, labels, batch_size):
    num_correct = torch.sum(torch.max(outputs, dim = 1)[1] == labels).item()
    return num_correct / batch_size

transform = transforms.Compose([transforms.Resize(128), transforms.CenterCrop(128), transforms.ToTensor()])
test_dataset = loader('test.csv','/datasets/cs154-fa19-public/',transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

def per_class_model_performance(performances):
    
    perf_df = []
    
    #for each class, calculate accuracy, recall, precision and bcr
    for i in range(len(performances)):
        
        cur_perf = performances[i]
        accu, recall, precision, bcr = get_per_class_values(cur_perf)
        
        perf_df.append([accu, precision, recall, bcr])
        
    #make it to a dataframe
    perf_df = pd.DataFrame(perf_df)
    perf_df.columns = ['Accuracy', 'Precision', 'Recall', 'BCR']
    
    return perf_df
 

#get measurement recall, accuracy, precision and bcr value for the current class
def get_per_class_values(cur_perf):    
    
    tp = cur_perf['TP']
    fp = cur_perf['FP']
    tn = cur_perf['TN']
    fn = cur_perf['FN']
    
    #accuracy
    accu = (tp + tn)/(tp + tn + fp + fn)
    
    #recall
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    
    #precision
    if fp + tp == 0:
        precision = 0
    else:
        precision = tp / (fp + tp)
        
    #bcr
    bcr = (precision + recall) / 2
    
    return accu, recall, precision, bcr


# In[ ]:


#Apply to test set and test Performances
all_class_performance = []
test_accuracies = []

#turn list into a sparse matrix
def make_matrix(lst):
    mat = []
    for i in lst:
        cur = [0] * 201
        cur[i] = 1
        mat.append(cur)
    return np.array(mat)


for i in range(201):
    all_class_performance.append({'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0})

with torch.no_grad(): 
    
    test_accu = 0
    
    for minibatch_count, (images, labels) in enumerate(test_loader, 0):
        
        print("current minibatch: ", minibatch_count)
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(device), labels.to(device) 
        outputs = model(images)
        
        test_accu += calculate_accu(outputs, labels, batch_size)
        
        predictions = torch.max(outputs, dim = 1)[1]

        pred = make_matrix(predictions.tolist())
        lab = make_matrix(labels.tolist())

        for i in range(pred.shape[0]):
            for j in range(201):
                if (pred[i][j] == 1) and (lab[i][j] == 1):
                    all_class_performance[j]['TP'] += 1
                elif (pred[i][j] == 1) and (lab[i][j] == 0):
                    all_class_performance[j]['FP'] += 1
                elif (pred[i][j] == 0) and (lab[i][j] == 0):
                    all_class_performance[j]['TN'] += 1
                else:
                    all_class_performance[j]['FN'] += 1
                    
    test_accuracies = (test_accu/minibatch_count)


# In[ ]:





# ## Visualizations

# In[ ]:


import matplotlib.pyplot as plt


# ### Training and Validation Loss Curves


# In[ ]:


print('The test accuracy is ' + str(test_accu/minibatch_count))

#from baseline_copy  The test accuracy is 0.6184375
#from baseline_copy1 The test accuracy is 0.6184375

/datasets/home/15/015/kzsidiqi/.conda/envs/pas/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/datasets/home/15/015/kzsidiqi/.conda/envs/pas/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/datasets/home/15/015/kzsidiqi/.conda/envs/pas/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm

current minibatch:  0
current minibatch:  1
current minibatch:  2
current minibatch:  3
current minibatch:  4
current minibatch:  5
current minibatch:  6
current minibatch:  7
current minibatch:  8
current minibatch:  9
current minibatch:  10
current minibatch:  11
current minibatch:  12
current minibatch:  13
current minibatch:  14
current minibatch:  15
current minibatch:  16
current minibatch:  17
current minibatch:  18
current minibatch:  19
current minibatch:  20
current minibatch:  21
current minibatch:  22
current minibatch:  23
current minibatch:  24
current minibatch:  25
current minibatch:  26
current minibatch:  27
current minibatch:  28
current minibatch:  29
current minibatch:  30
current minibatch:  31
current minibatch:  32
current minibatch:  33
current minibatch:  34
current minibatch:  35
current minibatch:  36
current minibatch:  37
current minibatch:  38
current minibatch:  39
current minibatch:  40
current minibatch:  41
current minibatch:  42
current minibatch:  4

In [6]:
df_fold2 = per_class_model_performance(all_class_performance)

In [7]:
df_fold2.head()

,Accuracy,Precision,Recall,BCR
0,1.000000,0.000000,0.000000,0.000000
1,0.990729,0.535714,0.468750,0.502232
2,0.993820,0.076923,0.111111,0.094017
3,0.997219,0.692308,0.642857,0.667582
4,0.992274,0.222222,0.266667,0.244444


In [3]:
df_fold1 = per_class_model_performance(all_class_performance)

In [4]:
df_fold1.head()

,Accuracy,Precision,Recall,BCR
0,1.000000,0.000000,0.000000,0.000000
1,0.985476,0.307692,0.375000,0.341346
2,0.996910,0.400000,0.222222,0.311111
3,0.996910,0.666667,0.571429,0.619048
4,0.994438,0.411765,0.466667,0.439216


In [20]:
top, bottom = get_values(df_avg)
print("  TOP 5 CLASSES for ZekeNet 2-fold X-Vldtn")
print(top)
print()
print("  BOT 5 CLASSES for ZekeNet 2-fold X-Vldtn")
print(bottom)

  TOP 5 CLASSES for ZekeNet 2-fold X-Vldtn
     Accuracy  Precision    Recall       BCR
199  0.999536   0.845238  0.916667  0.880952
91   0.998764   0.757143  0.642857  0.700000
133  0.998764   0.875000  0.625000  0.750000
47   0.998609   0.708333  0.950000  0.829167
84   0.998455   0.750000  0.562500  0.656250

  TOP 5 CLASSES for ZekeNet 2-fold X-Vldtn
     Accuracy  Precision    Recall       BCR
1    0.988103   0.421703  0.421875  0.421789
21   0.987330   0.369748  0.451613  0.410680
117  0.986712   0.168182  0.500000  0.334091
109  0.986712   0.284512  0.283333  0.283923
37   0.986557   0.356352  0.483333  0.419843


In [11]:
df_avg = pd.concat([df_fold1,df_fold2]).groupby(level=0).mean()

In [12]:
df_avg.head()

,Accuracy,Precision,Recall,BCR
0,1.000000,0.000000,0.000000,0.000000
1,0.988103,0.421703,0.421875,0.421789
2,0.995365,0.238462,0.166667,0.202564
3,0.997064,0.679487,0.607143,0.643315
4,0.993356,0.316993,0.366667,0.341830


In [8]:
def get_values(df):
    sorted_accs = df.sort_values(by="Accuracy", ascending=False)
    top_5 = sorted_accs[1:6]
    bot_5 = sorted_accs[-5:]
    return top_5, bot_5

In [ ]:
df = loader('test.csv','/datasets/cs154-fa19-public/')
df_tst = dataset.frame

classes = []
classes.append('no classes for label 0')
c = 0
for images, labels in df:
#     class_labels.append(labels)
#     print(dataset[0])
    
    print(images.filename.find('test/'))
    if c == 0:
        break
df_tst.head()

In [ ]:
for path in df_tst.path:
#     print(path)
    i = path.find('test/') + 5
    name = path[i:]
#     print(name)
    j = name.find('/')
    name = name[:j]
#     print(name)
    if not name in classes:
        classes.append(name)
        
df_tst.label.value_counts().to_dict()

In [ ]:
def report_metrics(best5, overall=False):
    
    #display average overall metrics
    if overall:
        dosomething
        
    
    if best5:
        #display metrics for top 5 classes
        hi
    else:
        #display metrics for worst 5 classes
        hi
        
    

In [ ]:
def update_score(t, y):
    '''
    t: ground truth
    y: predicted label
    '''
    TP[t]
    
    
    

In [ ]:
# print(len(test_loader))
classes_right = [0] * 201
# classes_right[0] = 1
# class_images = []
# for images, labels in dataset:
#     class_labels.append(labels)
# #     class_images.append(images)
    
# # print(np.amax(class_labels), np.amin(class_labels))
# # c = 0

# self.model = model 
# Since if the model is wrapped by the `DataParallel` class, you won't be able to access its attributes
# unless you write `model.module` which breaks the code compatibility. We use `model_attr_accessor` for attributes
# accessing only.
# model = model.module

classes_count = [0] * 201
# print(classes_count, len(classes_count))
# print(classes_count)



pred_classes = []
accu = 0
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader, 0):
        images, labels = images.to(device), labels.to(device)
        y = model(images)
        t = labels.item()
        classes_count[t] += 1
        y_i = torch.argmax(y).item()
        pred_classes.append(y_i)
#         print(i)
#         print(images)
#         print(labels)
#         print(labels.item())
        if t == y_i:
            accu += 1
            classes_right[t] += 1
            
# print(pred_classes)
print(np.amax(pred_classes), np.amin(pred_classes))
# # print(y)
# # print(torch.argmax(y).item())
print("accuracy: ", (accu/len(dataset)))
print(accu)
classes_accu = np.array(classes_right[1:]) / np.array(classes_count[1:])
#b = best, w = worst
b = np.argmax(classes_accu)
w = np.argmin(classes_accu)

print("best class performace with (accuracy): ", b+1, "(" + str(classes_accu[b]) + ")", np.amax(classes_accu), " --- " + str(classes_right[b+1]) + "/" + str(classes_count[b+1]) + " right")

print("worst class performace with (accuracy): ", w+1, "(" + str(classes_accu[w]) + ")", np.amin(classes_accu), " --- " + str(classes_right[w+1]) + "/" + str(classes_count[w+1]) + " right")